In [3]:
import urllib
import os,sys
import numpy as np
import pandas as pd

from sklearn import feature_extraction
from sklearn import preprocessing
from random import seed, shuffle
#from __future__ import division
#from collections import defaultdict
#import utils as ut

SEED = 1234
seed(SEED)
np.random.seed(SEED)

def check_data_file(fname):
    files = os.listdir(".") # get the current directory listing
    print("Looking for file '%s' in the current directory...",fname)

    if fname not in files:
        print("'%s' not found! Downloading from GitHub...",fname)
        addr = "https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv"
        response = urllib.request.urlopen(addr)
        data = response.read()
        fileOut = open(fname, "wb")
        fileOut.write(data)
        fileOut.close()
        print("'%s' download and saved locally..",fname)
    else:
        print("File found in current directory..")
    
COMPAS_INPUT_FILE = "compas-scores-two-years.csv"
check_data_file(COMPAS_INPUT_FILE)

Looking for file '%s' in the current directory... compas-scores-two-years.csv
File found in current directory..


In [4]:
# preparing data
df=pd.read_csv('compas-scores-two-years.csv')
df = df[df.columns[~df.isnull().any()]]
#df.nunique()
df = df[['sex','age_cat', 'race', 'juv_fel_count', 'decile_score', 'juv_misd_count', 'priors_count', 'c_charge_degree', 'is_recid', 
         'is_violent_recid', 'decile_score.1', 'score_text', 'v_score_text', 'priors_count.1', 'event', 'two_year_recid']]
#df

,sex,age_cat,race,juv_fel_count,decile_score,juv_misd_count,priors_count,c_charge_degree,is_recid,is_violent_recid,decile_score.1,score_text,v_score_text,priors_count.1,event,two_year_recid
0,Male,Greater than 45,Other,0,1,0,0,F,0,0,1,Low,Low,0,0,0
1,Male,25 - 45,African-American,0,3,0,0,F,1,1,3,Low,Low,0,1,1
2,Male,Less than 25,African-American,0,4,0,4,F,1,0,4,Low,Low,4,0,1
3,Male,Less than 25,African-American,0,8,1,1,F,0,0,8,High,Medium,1,0,0
4,Male,25 - 45,Other,0,1,0,2,F,0,0,1,Low,Low,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7209,Male,Less than 25,African-American,0,7,0,0,F,0,0,7,Medium,Medium,0,0,0
7210,Male,Less than 25,African-American,0,3,0,0,F,0,0,3,Low,Medium,0,0,0
7211,Male,Greater than 45,Other,0,1,0,0,F,0,0,1,Low,Low,0,0,0
7212,Female,25 - 45,African-American,0,2,0,3,M,0,0,2,Low,Low,3,0,0


In [15]:
# classifier
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
from sklearn.model_selection import train_test_split
import tensorflow as tf

transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))])


ground_truth = "two_year_recid"
y = df['two_year_recid']
x = df.drop([ground_truth], axis=1)

x_train, x_test, y_train, y_test = train_test_split( x, y,test_size=0.2) #0.2 meaning test set should be 20% of dataset

print('shape of x_train:', x_train.shape ,'\n')
print('shape of x_test:', x_test.shape ,'\n')
#print(x_train.head())
#print(x_test.head())

risk_label = ('one', 'two', 'three', 'four','five', 'six', 'seven', 'eight', 'nine', 'ten')
label = ('low', 'medium','high', 'risk')

shape of x_train: (5771, 15) 

shape of x_test: (1443, 15) 



In [13]:
input_size = 700 #flaten each image(28x28)
hidden_sizes = [300, 600]
output_size = 10

model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      nn.LogSoftmax(dim=1))
print(model)

Sequential(
  (0): Linear(in_features=700, out_features=300, bias=True)
  (1): ReLU()
  (2): Linear(in_features=300, out_features=600, bias=True)
  (3): ReLU()
  (4): Linear(in_features=600, out_features=10, bias=True)
  (5): LogSoftmax()
)


In [10]:
model(x_train, y_test, epoch = 4)

TypeError: forward() got an unexpected keyword argument 'epochs'